# Model Inference

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# !cp /content/drive/MyDrive/COCO_val2014_000000000042.jpg .
# !cp /content/drive/MyDrive/final_checkpoint.zip .
# !unzip final_checkpoint.zip -d final_checkpoint

Загрузим нашу модель

In [3]:
import torch
from PIL import Image

from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True, torch_dtype=torch.bfloat16)

model = model.to(device='cuda', dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True)
model.eval();

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-V-2:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpmv.py:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.20M [00:00<?, ?B/s]

И подгрузим ей наши собственные веса

In [4]:
# Базовый путь, куда сохранялись веса модели при обучении
base_path = "output/output_minicpmv2_lora/checkpoint-100/"

# Список весок(отвечающих за разные части модели) которые нужно подгрузить
weight_paths = [
    "vpm_resampler_embedtokens.pt",
    "global_step100/mp_rank_00_model_states.pt"
]

for weight_path in weight_paths:
    path_to_weights = base_path + weight_path
    own_weights = torch.load(path_to_weights)
    model.load_state_dict(own_weights, strict=False)

Теперь зададим вопрос к соответствующей картинке, и получим на него ответ

In [16]:
image_path = "data/source_data/all_images/COCO_val2014_000000000042.jpg" # здесь можно менять картинку, для которой будет вопрос

image = Image.open(image_path).convert('RGB') 
question = 'How many red shoes are there in this picture?' # здесь можно менять вопрос
msgs = [{'role': 'user', 'content': question}]

res, content, _ = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7
)

print(res)

There are two red shoes in the picture.
